In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_5%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.90,NaN,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0,1.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,NaN,3.18,7.19,...,234.51,170.0,169.0,0.01,NaN,2.25,0.0,0.00,23.0,1.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0,1.0
3,2.50,0.0,1.16,1.63,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,NaN,0.0,0.00,23.0,1.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0.0
1036,1.76,NaN,NaN,0.69,2.43,7.57,0.74,2.72,NaN,11.93,...,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0,0.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,NaN,14.24,...,149.64,79.0,77.0,0.01,NaN,20.00,1.0,4.57,1.0,0.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,451.37,75.0,70.0,0.01,0.0,NaN,3.0,19.58,1.0,0.0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         56
Jitter (local, absolute)               55
Jitter (rap)                           60
Jitter (ppq5)                          52
Jitter (ddp)                           59
Shimmer (local)                        37
Shimmer (local, dB)                    54
Shimmer (apq3)                         44
Shimmer (apq5)                         71
Shimmer (apq11)                        60
Shimmer (dda)                          51
AC                                     50
NTH                                    51
HTN                                    59
Median pitch                           54
Mean pitch                             69
Standard deviation                     52
Minimum pitch                          53
Maximum pitch                          52
Number of pulses                       57
Number of periods                      46
Mean period                            56
Standard deviation of period           53
Fraction of locally unvoiced frame

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 1456
persentase : 5.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         5.384615
Jitter (local, absolute)               5.288462
Jitter (rap)                           5.769231
Jitter (ppq5)                          5.000000
Jitter (ddp)                           5.673077
Shimmer (local)                        3.557692
Shimmer (local, dB)                    5.192308
Shimmer (apq3)                         4.230769
Shimmer (apq5)                         6.826923
Shimmer (apq11)                        5.769231
Shimmer (dda)                          4.903846
AC                                     4.807692
NTH                                    4.903846
HTN                                    5.673077
Median pitch                           5.192308
Mean pitch                             6.634615
Standard deviation                     5.000000
Minimum pitch                          5.096154
Maximum pitch                          5.000000
Number of pulses                       5.480769
Number of periods                      4

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",euc_dist[col])

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter
def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("max_count:",max_count)
#     print("max_vals:",max_vals)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count == 1 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None
    
def Final(df,col,k):
    terdekat = []
    mean = 0
    std = 0
    diffsquared = 0
    sum_diffsquared = 0
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        mean += y  
        euc_val.append(y)
    mean /= len(terdekat)
    
    print(euc_val)
    
    for x,y in terdekat:
        diffsquared = (y-mean)**2
        sum_diffsquared = diffsquared + sum_diffsquared
    std_dev = ((sum_diffsquared)/len(terdekat))**(1/2)
    print("mean : ",mean)
    print("standar deviasi : ",std_dev)
    
    euc_val.sort()
    fuzzy = hitung(euc_val,5)
    
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,gaussian(y,mean,std_dev)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def gaussian(val, mean, std_dev):
    gaussian_mf = 2.718**((-1 * (val-mean)**2) / (2 * std_dev**2))
    return gaussian_mf

In [11]:
data = df.iloc[:, :-1]
df2 = data.copy()
dummy = df2.fillna(0)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27
0,1.49,0.0,0.90,0.00,2.70,8.33,0.78,4.52,4.61,6.80,...,142.23,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,0.00,3.18,7.19,...,159.52,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,146.45,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0
3,2.50,0.0,1.16,1.63,3.47,13.51,0.00,5.26,8.77,16.78,...,182.71,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,182.82,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,96.15,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0
1036,1.76,0.0,0.00,0.69,2.43,7.57,0.74,2.72,0.00,11.93,...,121.49,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,0.00,14.24,...,95.92,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,90.05,451.37,75.0,70.0,0.01,0.0,0.00,3.0,19.58,1.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 4642


In [13]:
euc_dist = {}

In [14]:
for i in range(dummy.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,5)

Kolom : 1
[1.21, 1.42, 3.09, 2.72, 1.17]
mean :  1.922
standar deviasi :  0.815534180767428
fuzzy :  [(1.21, 0.6831325480750221), (1.42, 0.8274308834638313), (3.09, 0.35862502334044766), (2.72, 0.6195999487751875), (1.17, 0.653713873142807)]
fuzzy :  None
pembilang :  5.559850652056994
penyebut :  3.142502276797296
rata-rata tertimbang :  1.769243157947162
 
[2.08, 0.0, 2.75, 1.5, 1.54]
mean :  1.574
standar deviasi :  0.908088101452717
fuzzy :  [(2.08, 0.8562199451023982), (0.0, 0.22267777995494548), (2.75, 0.4323747873243951), (1.5, 0.9966855480948584), (1.54, 0.9992993927022166)]
fuzzy :  None
pembilang :  6.003917537858776
penyebut :  3.507257453178814
rata-rata tertimbang :  1.7118553793127178
 
[0.56, 0.61, 0.46, 0.29, 0.28]
mean :  0.44000000000000006
standar deviasi :  0.1354990774876346
fuzzy :  [(0.56, 0.6756256001863415), (0.61, 0.455228661429763), (0.46, 0.9891669875893309), (0.29, 0.5418957896699433), (0.28, 0.4980297470742016)]
fuzzy :  None
pembilang :  1.407654742052658

[0.0, 7.46, 3.9, 2.63, 5.12]
mean :  3.822
standar deviasi :  2.4881511208123994
fuzzy :  [(0.0, 0.30738659097123705), (7.46, 0.34342002954479933), (3.9, 0.9995088049450256), (2.63, 0.8915957787185731), (5.12, 0.8727928932831712)]
fuzzy :  None
pembilang :  13.273594271329486
penyebut :  3.4147040974628062
rata-rata tertimbang :  3.887187262050622
 
[0.0, 3.0, 1.59, 3.63, 1.63]
mean :  1.9699999999999995
standar deviasi :  1.2615387429643212
fuzzy :  [(0.0, 0.2954819686172821), (3.0, 0.7165755842076675), (1.59, 0.9556515945314067), (3.63, 0.4207799458155708), (1.63, 0.9643368366321917)]
fuzzy :  None
pembilang :  6.768513034948933
penyebut :  3.3528259298041188
rata-rata tertimbang :  2.01874871426575
 
[0.0, 3.04, 1.96, 1.3, 3.45]
mean :  1.95
standar deviasi :  1.2379983844900608
fuzzy :  [(0.0, 0.28927486414288184), (3.04, 0.6787115976779379), (1.96, 0.9999673804674429), (1.3, 0.8712557015076822), (3.45, 0.4800085065895784)]
fuzzy :  None
pembilang :  6.811881082351151
penyebut :  3

[0.0, 5.29, 2.93, 1.26, 1.2961014310229249]
mean :  2.155220286204585
standar deviasi :  1.822723594468498
fuzzy :  [(0.0, 0.49709053929406427), (5.29, 0.22792036510450814), (2.93, 0.9136281529526588), (1.26, 0.8863894621372724), (1.2961014310229249, 0.8948776911553074)]
fuzzy :  None
pembilang :  6.159332197943987
penyebut :  3.419906210643811
rata-rata tertimbang :  1.801023717777473
 
[0.0, 1.55, 2.8, 2.98, 4.53]
mean :  2.372
standar deviasi :  1.5173582306100297
fuzzy :  [(0.0, 0.2947174939838999), (1.55, 0.863534846784165), (2.8, 0.9610033462708161), (2.98, 0.9228666919665895), (4.53, 0.3637698609372127)]
fuzzy :  None
pembilang :  8.42730859417975
penyebut :  3.405892239942683
rata-rata tertimbang :  2.4743321281126534
 
[0.0, 2.67, 1.28, 1.73, 2.06]
mean :  1.548
standar deviasi :  0.8969147116643811
fuzzy :  [(0.0, 0.2255440831938884), (2.67, 0.45732400437685744), (1.28, 0.9563448405609828), (1.73, 0.9796247476805516), (2.06, 0.8496629201926607)]
fuzzy :  None
pembilang :  5.8

[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0,

[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0]
mean :  0.0
standar deviasi :  0.0
median :  3
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0,

...

[0.0, 0.0, 2.9367522340054855, 0.03705525933399557, 2.9440865791453312]
mean :  1.1835788144969626
standar deviasi :  1.4345200250295569
fuzzy :  [(0.0, 0.7115322018462614), (0.0, 0.7115322018462614), (2.9367522340054855, 0.4739154325757735), (0.03705525933399557, 0.7266154247642954), (2.9440865791453312, 0.47095757358585755)]
fuzzy :  None
pembilang :  2.8052370000882743
penyebut :  3.094552834618449
rata-rata tertimbang :  0.9065080320188339
 
[0.0, 0.03705525933399557, 0.0, 0.0017130227748099233, 0.0]
mean :  0.0077536564217610985
standar deviasi :  0.014665815707304504
fuzzy :  [(0.0, 0.8695829290030289), (0.03705525933399557, 0.13591846180166436), (0.0, 0.8695829290030289), (0.0017130227748099233, 0.9186813539701998), (0.0, 0.8695829290030289)]
fuzzy :  None
pembilang :  0.006610215932482612
penyebut :  3.663348602780951
rata-rata tertimbang :  0.001804419030027503
 
[0.0, 9.732472766920079, 25.13, 33.34, 32.55]
mean :  20.150494553384014
standar deviasi :  13.16878029605663
fuzzy

[0.0, 0.0013684532532418337, 0.0017130227748099233, 0.03705525933399557, 0.001804419030027503]
mean :  0.008388230878414966
standar deviasi :  0.014348123203617575
fuzzy :  [(0.0, 0.8429279271903387), (0.0013684532532418337, 0.8872140986474826), (0.0017130227748099233, 0.8974397851726311), (0.03705525933399557, 0.13591543258844754), (0.001804419030027503, 0.9000847452465871)]
fuzzy :  None
pembilang :  0.009411957455654926
penyebut :  3.663581988845487
rata-rata tertimbang :  0.0025690587748033278
 
[0.0, 0.001804419030027503, 0.03705525933399557, 0.0, 30.25]
mean :  6.057771935672805
standar deviasi :  12.096122289602128
fuzzy :  [(0.0, 0.88215392413038), (0.001804419030027503, 0.8822198125135581), (0.03705525933399557, 0.8835040416842297), (0.0, 0.88215392413038), (30.25, 0.13536372001316044)]
fuzzy :  None
pembilang :  4.1290828960037125
penyebut :  3.665395422471708
rata-rata tertimbang :  1.1265040793932468
 
[0.0, 0.03705525933399557, 0.001804419030027503, 0.0, 0.0554626315753532

[0.0, 20.44, 2.156963836698081, 1.0649090144097777, 16.66]
mean :  8.06437457022157
standar deviasi :  8.671390178715438
fuzzy :  [(0.0, 0.6489476684151271), (20.44, 0.36120331960322993), (2.156963836698081, 0.7929233047441846), (1.0649090144097777, 0.7219895315696883), (16.66, 0.6118611472104495)]
fuzzy :  None
pembilang :  20.0557626193025
penyebut :  3.1369249715426792
rata-rata tertimbang :  6.393446703776107
 
Kolom : 27
[1.0, 1.0, 1.0, 8.0, 1.0]
mean :  2.4
standar deviasi :  2.8
fuzzy :  [(1.0, 0.8825083402897205), (1.0, 0.8825083402897205), (1.0, 0.8825083402897205), (8.0, 0.13536335042701722), (1.0, 0.8825083402897205)]
fuzzy :  None
pembilang :  4.612940164575019
penyebut :  3.665396711585899
rata-rata tertimbang :  1.2585104771862876
 
[8.0, 8.0, 5.0, 1.0, 8.0]
mean :  6.0
standar deviasi :  2.756809750418044
fuzzy :  [(8.0, 0.7686414989480145), (8.0, 0.7686414989480145), (5.0, 0.9363343522475271), (1.0, 0.19309628880484983), (8.0, 0.7686414989480145)]
fuzzy :  None
pembilan

[0.0, 1.0, 1.0, 1.0, 55.0]
mean :  11.6
standar deviasi :  21.703455946000858
fuzzy :  [(0.0, 0.8669117710178962), (1.0, 0.8875808736884427), (1.0, 0.8875808736884427), (1.0, 0.8875808736884427), (55.0, 0.13544958031488752)]
fuzzy :  None
pembilang :  10.112469538384142
penyebut :  3.665103972398112
rata-rata tertimbang :  2.7591221462040703
 
[0.0, 1.0, 1.0, 1.0, 1.0]
mean :  0.8
standar deviasi :  0.4000000000000001
fuzzy :  [(0.0, 0.13536335042701728), (1.0, 0.8825083402897206), (1.0, 0.8825083402897206), (1.0, 0.8825083402897206), (1.0, 0.8825083402897206)]
fuzzy :  None
pembilang :  3.5300333611588823
penyebut :  3.6653967115858994
rata-rata tertimbang :  0.9630699318305304
 
[0.0, 55.0, 1.0, 1.0, 1.0]
mean :  11.6
standar deviasi :  21.70345594600086
fuzzy :  [(0.0, 0.8669117710178962), (55.0, 0.13544958031488757), (1.0, 0.8875808736884427), (1.0, 0.8875808736884427), (1.0, 0.8875808736884427)]
fuzzy :  None
pembilang :  10.112469538384143
penyebut :  3.665103972398112
rata-rata 

[0.0, 26.0, 2.4749256821599985, 20.0, 20.0]
mean :  13.694985136432
standar deviasi :  10.434195394919035
fuzzy :  [(0.0, 0.4226326743590923), (26.0, 0.49892546867076487), (2.4749256821599985, 0.5609673744388868), (20.0, 0.8331438842843861), (20.0, 0.8331438842843861)]
fuzzy :  None
pembilang :  47.686170118667995
penyebut :  3.148813286037516
rata-rata tertimbang :  15.14417203780175
 
[0.0, 1.0, 5.0, 5.0, 1.0]
mean :  2.4
standar deviasi :  2.154065922853802
fuzzy :  [(0.0, 0.5376081663969953), (1.0, 0.8096242743823724), (5.0, 0.48269381620953977), (5.0, 0.48269381620953977), (1.0, 0.8096242743823724)]
fuzzy :  None
pembilang :  6.446186710860142
penyebut :  3.1222443475808195
rata-rata tertimbang :  2.0646003301614697
 


In [15]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27
0,1.49,0.0,0.900000,0.317923,2.70,8.33,0.780000,4.520000,4.61000,6.800000,...,142.23,187.580000,160.0,159.0,0.01,0.0,8.114101,0.07386,4.007126,23.0
1,0.73,0.0,0.350000,0.380000,1.06,5.86,0.640000,4.446322,3.18000,7.190000,...,159.52,234.510000,170.0,169.0,0.01,0.0,2.250000,1.00000,5.343241,23.0
2,1.22,0.0,0.730000,0.670000,2.20,8.72,0.880000,4.350000,5.17000,7.550000,...,146.45,211.440000,1431.0,1427.0,0.01,0.0,10.660000,1.00000,0.180000,23.0
3,2.50,0.0,1.160000,1.630000,3.47,13.51,1.046904,5.260000,8.77000,16.780000,...,182.71,220.230000,94.0,92.0,0.01,0.0,17.596929,1.00000,0.180000,23.0
4,3.51,0.0,1.720000,1.540000,5.15,9.11,1.040000,3.100000,4.93000,12.820000,...,182.82,225.160000,117.0,114.0,0.01,0.0,18.180000,1.00000,13.320000,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.720000,2.510000,8.16,9.85,0.970000,4.590000,6.81000,8.300000,...,96.15,158.590000,63.0,60.0,0.01,0.0,24.190000,2.00000,15.110000,1.0
1036,1.76,0.0,1.003714,0.690000,2.43,7.57,0.740000,2.720000,5.54116,11.930000,...,121.49,143.800000,47.0,44.0,0.01,0.0,46.050000,1.00000,25.790000,1.0
1037,2.91,0.0,1.140000,1.270000,3.42,15.49,1.460000,6.430000,7.29871,14.240000,...,95.92,149.640000,79.0,77.0,0.01,0.0,20.000000,1.00000,4.570000,1.0
1038,3.79,0.0,1.790000,1.480000,5.36,15.70,1.370000,6.370000,11.09000,18.820000,...,90.05,451.370000,75.0,70.0,0.01,0.0,18.210189,3.00000,19.580000,1.0


In [16]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1040.000000,1.040000e+03,1040.00000,1040.000000,1040.000000,1040.000000,1040.000000
mean,2.639291,0.0,1.236693,1.328203,3.686120,12.849255,1.184228,5.663279,7.838052,12.066649,...,134.382954,233.654544,109.216056,105.567546,7.838917e-03,0.00001,28.768150,1.286213,14.392649,12.763442
std,1.750320,0.0,0.970896,1.133440,2.907807,5.389795,0.413289,2.986688,4.763787,5.797608,...,46.584651,121.150199,149.771251,149.043156,3.956335e-03,0.00031,19.672152,1.503259,14.101689,15.681030
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,67.960000,85.540000,5.000000,4.000000,5.010356e-11,0.00000,0.088547,0.000050,0.001368,0.963070
25%,1.500000,0.0,0.620000,0.650000,1.800000,9.385000,0.940000,3.717500,5.157500,8.217253,...,101.240000,143.192500,42.000000,40.000000,9.630699e-03,0.00000,11.520000,0.305320,3.133273,1.000000
50%,2.350000,0.0,1.035000,1.110000,3.065000,12.215000,1.170000,5.120000,6.940000,11.300000,...,127.230000,194.825000,64.000000,61.500000,1.000000e-02,0.00000,27.436441,1.000000,9.741236,4.360457
75%,3.352500,0.0,1.590000,1.660000,4.760000,15.397500,1.400000,6.860000,9.235000,14.905000,...,158.840000,262.507500,112.250000,109.000000,1.000000e-02,0.00000,42.637500,1.327898,22.540000,23.000000
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,42.000000,...,452.080000,597.970000,1490.000000,1489.000000,1.000000e-02,0.01000,88.160000,12.000000,69.120000,55.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.values
y = df.iloc[:, -1].values

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=45)

In [23]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [24]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[129   6]
 [  3 122]]
0.9653846153846154
